In [2]:
#                                                   LIBRARIES

import os

import csv

# Dataset
from kaggle.api.kaggle_api_extended import KaggleApi
from datasets import load_dataset

# Image preprocesssing- processing
import cv2
from PIL import Image
import numpy as np

# Neural Network libraries
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout

import re

In [3]:
IMG_EXTENSIONS = ['jpg','JPG','png','PNG']

In [4]:
#                                                 FUNCIONES AUXILIARES


#                                       getFilesWithAbsolutePath(path, ext = '')

# Dado un directorio devuelve una lista con el path absoluto de todos los ficheros con extensión ext

def getFilesWithAbsolutePath(path, ext = ''):
    # to store files in a list
    list = []

    if bool(ext):
        ext = '.' + ext

    # dirs=directories
    for (root, folder, files) in os.walk(path):
        for file in files:
            filePath = os.path.abspath(os.path.join(root, file))
            if ext in filePath:
                list.append(filePath)
                #print(filePath)

    return list

#lista = getFilesWithAbsolutePath(mypath, ext='pdf')
#print(len(lista), lista[:3])


#                            getFilesWithAbsolutePathListaExtensiones(path, extList = [''])

# Dado un directorio devuelve una lista con el path absoluto de
# todos los ficheros con extensión en extList

def getFilesWithAbsolutePathListaExtensiones(path):
    # to store files in a list
    l_list = []
    extList = IMG_EXTENSIONS

    if len(extList) == 0:
        extList = ['']

    for ext in extList:
        l_list += getFilesWithAbsolutePath(path, ext)

    return l_list


# lista = getFilesWithAbsolutePathListaExtensiones(mypath, extList = ['epub','pdf'])
# print(len(lista))

#                       FUNCIONES AUXILIARES PARA ORDENAR LOS NOMBRES DE LAS IMÁGENES DE MODO HUMANO
#                                        Modo humano = por orden alfanumerico

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [9]:
def create_CSV():
    INPUT_FOLDER = "../datasets/animals/data"
    INPUT_FOLDER = os.path.abspath(INPUT_FOLDER) + os.sep
    csvArray = np.empty([len(getFilesWithAbsolutePathListaExtensiones(INPUT_FOLDER)), 2], dtype=object)

    csvOutput = "imgs_and_labels_list" + '.csv'

    idx_aux = 0

    for animal in os.listdir(INPUT_FOLDER):
        #print(animal)
        animal_imgs = getFilesWithAbsolutePathListaExtensiones(INPUT_FOLDER + animal)
        animal_imgs.sort(key=natural_keys)
        #print(animal_imgs)

        for img in animal_imgs:
            csvArray[idx_aux,:] = [img, animal]
            idx_aux += 1



    try:
        np.savetxt(csvOutput, csvArray, delimiter=",",fmt='%s')
        print()
        print('CSV salvado: ', csvOutput)
    except:
        print('ERROR ESCRITURA FINAL CSV:', csvOutput)


CSV salvado:  imgs_and_labels_list.csv


In [74]:
create_CSV()


CSV salvado:  imgs_and_labels_list.csv


In [ ]:
def load_and_save_data_from_dir():



